<a href="https://colab.research.google.com/github/KANG-HYUNIL/Kaggle_Competition/blob/main/Kaggle_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# kaggle api key upload
from google.colab import files
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kangcon","key":"c481bb33ab428580ff30cafdaccdb05d"}'}

In [ ]:
# directory for kaggle
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!pwd


-rw-r--r-- 1 root root 63 Sep  3 12:40 kaggle.json
/content


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#download titanic competition dataset
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 95.6MB/s]


In [ ]:
#unzip dataset
!unzip titanic.zip


Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
#analysis train and test data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_csv("train.csv")

#상위 5행
df.head()




,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
import matplotlib.pyplot as plt

#Age 부터 분석 시작
df["Age"].isna().sum()


np.int64(177)

In [ ]:
df2 = pd.read_csv("test.csv")
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [ ]:
## 결측치 처리 및 인코딩 하는 메서드와 코드 !! ##
## ~~~~~~~~~~~~~~~~~!!!!!!!!!!!!!!!!!!~~~~~~~~~~~~~~~33##

# --- 0) 아티팩트(구간 경계 + 티켓카운트 맵) 생성 ---
def build_artifacts(df_train_raw, df_test_raw, n_age_bins=5, n_fare_bins=5):
    # 1) train 기준 등빈도 경계(qcut) 추출
    age_edges  = pd.qcut(df_train_raw['Age'].dropna(),  q=n_age_bins, retbins=True, duplicates='drop')[1]
    fare_edges = pd.qcut(df_train_raw['Fare'].dropna(), q=n_fare_bins, retbins=True, duplicates='drop')[1]

    # 2) 경계 안전장치: 양 끝을 무한대로 확장 (test에서 범위 밖 값 방지)
    #    (첫/끝값만 -inf, +inf로 바꿔줍니다)
    age_edges  = age_edges.copy()
    fare_edges = fare_edges.copy()
    age_edges[0],  age_edges[-1]  = -np.inf, np.inf
    fare_edges[0], fare_edges[-1] = -np.inf, np.inf

    # 3) TicketCount: train+test 합쳐 카운트(캐글 튜토리얼 스타일)
    both = pd.concat([df_train_raw[['Ticket']], df_test_raw[['Ticket']]], axis=0, ignore_index=True)
    ticket_counts = both['Ticket'].value_counts()  # Series: index=Ticket, values=count

    return {
        'age_edges': age_edges,
        'fare_edges': fare_edges,
        'ticket_counts': ticket_counts
    }

# --- 1) 전처리 ---
def preprocess(df, artifacts=None, drop_pid=True):
    df = df.copy()

    # ============ Age ============
    df['AgeMissing'] = df['Age'].isna().astype(int)

    # Title 추출 및 정리
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+).', expand=False)
    main_titles = ['Mr','Miss','Mrs','Master']
    df['Title'] = df['Title'].where(df['Title'].isin(main_titles), 'Rare')

    # Title별 평균 나이로 결측치 채우기
    title_age_map = df.groupby('Title')['Age'].mean()
    df['Age'] = np.where(df['Age'].isna(), df['Title'].map(title_age_map), df['Age'])

    # Title → One-hot
    df = pd.concat([df, pd.get_dummies(df['Title'], prefix='Title')], axis=1)

    # ============ Sex ============
    df = pd.concat([df, pd.get_dummies(df['Sex'], prefix='Sex')], axis=1)

    # ============ Embarked ============
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

    # ============ Fare ============
    if df['Fare'].isna().sum() > 0:
        df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

    # ============ Family Features ============
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone']    = (df['FamilySize'] == 1).astype(int)
    # ⛔️ FarePerPerson 제거 (요청 반영)

    # ============ TicketCount ============
    if artifacts is not None and 'ticket_counts' in artifacts:
        df['TicketCount'] = df['Ticket'].map(artifacts['ticket_counts']).fillna(1).astype(int)
    else:
        df['TicketCount'] = df.groupby('Ticket')['Ticket'].transform('count').astype(int)

    # ============ Age/Fare Binning (train 경계 재사용) ============
    # artifacts가 있으면 pd.cut + 저장한 경계 재사용 (일관성 보장)
    if artifacts is not None and 'age_edges' in artifacts:
        df['AgeBin'] = pd.cut(
            df['Age'], bins=artifacts['age_edges'], include_lowest=True, labels=False
        )
    else:
        df['AgeBin'] = pd.qcut(df['Age'], q=5, labels=False, duplicates='drop')

    if artifacts is not None and 'fare_edges' in artifacts:
        df['FareBin'] = pd.cut(
            df['Fare'], bins=artifacts['fare_edges'], include_lowest=True, labels=False
        )
    else:
        df['FareBin'] = pd.qcut(df['Fare'], q=5, labels=False, duplicates='drop')

    # 결측 bin은 -1로 라벨링
    df['AgeBin']  = df['AgeBin'].astype('Int64').fillna(-1).astype(int)
    df['FareBin'] = df['FareBin'].astype('Int64').fillna(-1).astype(int)

    # ============ (선택) 표준화 ============
    # 모델 파이프라인에서 스케일링 쓸 거면 여기선 생략해도 됩니다.
    # 요청 반영: FarePerPerson 제거했으니 스케일링도 Age/Fare 정도만 필요시 수행
    for col in ['Age', 'Fare']:
        mean, std = df[col].mean(), df[col].std()
        df[col] = 0.0 if std == 0 else (df[col] - mean) / std

    # ============ Cabin ============
    df['Cabin']    = df['Cabin'].fillna('U')
    df['HasCabin'] = (df['Cabin'] != 'U').astype(int)
    df['Deck']     = df['Cabin'].str[0]
    df = pd.concat([df, pd.get_dummies(df['Deck'], prefix='Deck')], axis=1)

    # ============ 불필요 컬럼 제거 ============
    drop_cols = ['Name','Ticket','Cabin','Deck','Sex','Embarked','Title']
    if drop_pid and 'PassengerId' in df.columns:
        drop_cols.append('PassengerId')
    df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)

    # 중복 컬럼 제거
    df = df.loc[:, ~df.columns.duplicated()]
    return df

# === 사용 예시 ===
# df_raw, df2_raw: 원본 데이터프레임
art = build_artifacts(df, df2, n_age_bins=5, n_fare_bins=5)
df2_pid = df2['PassengerId'].copy()

df  = preprocess(df,  artifacts=art, drop_pid=True)   # train (Survived 포함)
df2 = preprocess(df2, artifacts=art, drop_pid=True)   # test  (Survived 없음)

KeyError: "None of [Index(['Ticket'], dtype='object')] are in the [columns]"

In [ ]:
# ✅ 1. train(df), test(df2) 컬럼 구조 비교
print("Train shape:", df.shape)
print("Test  shape:", df2.shape)

print("\n컬럼 차이 확인:")
print("Train에만 있는 컬럼:", set(df.columns) - set(df2.columns))
print("Test 에만 있는 컬럼:", set(df2.columns) - set(df.columns))

Train shape: (891, 32)
Test  shape: (418, 30)

컬럼 차이 확인:
Train에만 있는 컬럼: {'Survived', 'Deck_T'}
Test 에만 있는 컬럼: set()


In [ ]:
#훈련 데이터셋, 테스트 데이터 셋 최종 비교

def align_columns(train_df, test_df):
    """
    train_df를 기준으로 test_df의 컬럼을 맞춤
    - Survived는 test에 추가하지 않음
    - train에만 있는 컬럼은 test에 0으로 추가
    - test에만 있는 컬럼은 제거
    - 순서 train 기준으로 맞춤
    """
    train_cols = set(train_df.columns) - {'Survived'}  # 타깃은 제외
    test_cols  = set(test_df.columns)

    missing_in_test = train_cols - test_cols
    for col in missing_in_test:
        test_df[col] = 0

    extra_in_test = test_cols - train_cols
    test_df = test_df.drop(columns=extra_in_test)

    # 순서 train 기준 (Survived 제외)
    ordered_cols = [c for c in train_df.columns if c != 'Survived']
    test_df = test_df[ordered_cols]

    return test_df

# 사용 예시
df2 = align_columns(df, df2)

print("Train:", df.shape)
print("Test :", df2.shape)
print("최종 검증:", list(df.columns.drop('Survived')) == list(df2.columns))

Train: (891, 32)
Test : (418, 31)
최종 검증: True


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier




# 입력/타깃 분리
X = df.drop('Survived', axis=1)
y = df['Survived'].astype(int)

# # 학습/검증 분리 (층화 추출)
# X_train, X_val, y_train, y_val = train_test_split(
#     X, y, test_size=0.2, stratify=y
# )

# X_train.shape, X_val.shape


In [ ]:
# == 특성 공학 및 표준화 처리 후에 4개의 모델에 대한 재학습 후 평가 비교 ==

# === 모델 정의 (베이스라인 + 추가 앙상블) ===
models = {
    "LogisticRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=1000, n_jobs=None))  # n_jobs 옵션 없는 구현도 있어서 명시X
    ]),
    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=5))
    ]),
    "DecisionTree": DecisionTreeClassifier(random_state=42),

    # 기존 RandomForest 유지
    "RandomForest": RandomForestClassifier(
        n_estimators=200, random_state=42, n_jobs=-1
    ),

    # 새로 추가
    "ExtraTrees": ExtraTreesClassifier(
        n_estimators=300, random_state=42, n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingClassifier(
        n_estimators=200, learning_rate=0.05, max_depth=3, random_state=42
    ),
    "HistGradientBoosting": HistGradientBoostingClassifier(
        max_iter=300, learning_rate=0.05, max_depth=None, random_state=42
    ),
}

# === 5-Fold Stratified CV 설정 ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(accuracy_score)

# === 교차검증 실행 ===
rows = []
per_fold_scores = {}
for name, model in models.items():
    scores = cross_validate(
        model, X, y,
        cv=cv,
        scoring=scorer,
        return_train_score=True,
        n_jobs=-1
    )
    tr = scores['train_score']; va = scores['test_score']
    rows.append({
        "Model": name,
        "TrainAcc(mean)": tr.mean(),
        "TrainAcc(std)":  tr.std(),
        "ValAcc(mean)":   va.mean(),
        "ValAcc(std)":    va.std(),
        "Gap(mean)":      (tr - va).mean()
    })
    per_fold_scores[name] = {"train": tr, "val": va}

# === 표 정리/출력 ===
cv_df = pd.DataFrame(rows).sort_values("ValAcc(mean)", ascending=False)
display(cv_df.style.format({
    "TrainAcc(mean)": "{:.4f}",
    "TrainAcc(std)":  "{:.4f}",
    "ValAcc(mean)":   "{:.4f}",
    "ValAcc(std)":    "{:.4f}",
    "Gap(mean)":      "{:+.4f}",
}))

,Model,TrainAcc(mean),TrainAcc(std),ValAcc(mean),ValAcc(std),Gap(mean)
5,GradientBoosting,0.9150,0.0064,0.8339,0.0211,+0.0811
0,LogisticRegression,0.8420,0.0063,0.8283,0.0223,+0.0138
6,HistGradientBoosting,0.9820,0.0038,0.8204,0.0318,+0.1617
3,RandomForest,0.9888,0.0020,0.8092,0.0230,+0.1796
1,KNN,0.8563,0.0064,0.8069,0.0253,+0.0494
4,ExtraTrees,0.9888,0.0020,0.8069,0.0288,+0.1819
2,DecisionTree,0.9888,0.0020,0.7890,0.0302,+0.1997


In [ ]:
#Bayesian Optimization 활용한 Logistic Regression Model의 Hyper Parameter 최적화 연습 시도
!pip install -q scikit-optimize


from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args


# === 5-Fold Stratified CV 설정 ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(accuracy_score)

# === 탐색 공간 정의 ===
# - learning_rate: 부스팅 학습률 (log-uniform 권장: 1e-3 ~ 0.3)
# - n_estimators: 트리 개수 (50~500)
# - max_depth: 약한 트리 깊이 (1~5)
# - subsample: 각 트리 학습에 사용할 샘플 비율 (0.5~1.0)
# - min_samples_leaf: 리프 최소 샘플 (1~20)
# - max_features: 분할에 사용할 특성 수 (None, 'sqrt', 'log2')
space = [
    Real(1e-3, 3e-1, prior='log-uniform', name='learning_rate'),
    Integer(50, 500, name='n_estimators'),
    Integer(1, 5, name='max_depth'),
    Real(0.5, 1.0, name='subsample'),
    Integer(1, 20, name='min_samples_leaf'),
    Categorical([None, 'sqrt', 'log2'], name='max_features')
]

@use_named_args(space)
def objective(learning_rate, n_estimators, max_depth, subsample, min_samples_leaf, max_features):
    clf = GradientBoostingClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        subsample=subsample,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    # 교차검증: train/val 점수 수집
    scores = cross_validate(
        clf, X, y,
        cv=cv,
        scoring=scorer,
        return_train_score=True,
        n_jobs=-1
    )
    tr = scores['train_score']; va = scores['test_score']
    tr_mean, va_mean, gap_mean = tr.mean(), va.mean(), (tr - va).mean()

    print(
        f"[Trial] lr={learning_rate:.4g}  n_est={n_estimators:3d}  "
        f"depth={max_depth}  subsample={subsample:.2f}  leaf={min_samples_leaf:2d}  "
        f"max_feat={max_features}  |  train={tr_mean:.4f}  val={va_mean:.4f}  gap={gap_mean:+.4f}"
    )

    # maximize val(mean) → minimize(-val)
    return -va_mean

# === 최적화 실행 ===
result = gp_minimize(
    func=objective,
    dimensions=space,
    acq_func='EI',          # Expected Improvement
    n_calls=50,             # 예: 40~80 정도로 시작
    n_initial_points=12,
    random_state=42
)

# === 최적 결과 출력 ===
best_val_acc = -result.fun
(best_lr, best_n_est, best_depth, best_subsample, best_leaf, best_maxfeat) = result.x

print("\n=== Best (CV val mean acc) ===")
print(f"val_mean_acc : {best_val_acc:.4f}")
print("params:")
print(f"  learning_rate  : {best_lr}")
print(f"  n_estimators   : {best_n_est}")
print(f"  max_depth      : {best_depth}")
print(f"  subsample      : {best_subsample}")
print(f"  min_samples_leaf: {best_leaf}")
print(f"  max_features   : {best_maxfeat}")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.5 MB/s eta 0:00:00
[Trial] lr=0.094  n_est=133  depth=4  subsample=0.80  leaf= 9  max_feat=None  |  train=0.9388  val=0.8316  gap=+0.1072
[Trial] lr=0.01373  n_est=200  depth=2  subsample=0.83  leaf= 2  max_feat=log2  |  train=0.8221  val=0.8182  gap=+0.0039
[Trial] lr=0.2113  n_est= 50  depth=5  subsample=0.81  leaf=13  max_feat=None  |  train=0.9377  val=0.8406  gap=+0.0971
[Trial] lr=0.001141  n_est=286  depth=3  subsample=0.52  leaf=20  max_feat=None  |  train=0.7840  val=0.7834  gap=+0.0006
[Trial] lr=0.001677  n_est=328  depth=3  subsample=0.99  leaf=10  max_feat=log2  |  train=0.7854  val=0.7867  gap=-0.0014
[Trial] lr=0.04844  n_est=253  depth=1  subsample=0.97  leaf=12  max_feat=sqrt  |  train=0.8364  val=0.8294  gap=+0.0070
[Trial] lr=0.001095  n_est=154  depth=2  subsample=0.84  leaf=13  max_feat=log2  |  train=0.6162  val=0.6162  gap=-0.0000
[Trial] lr=0.002688  n_est=226  depth=2  subsample=0.88  leaf= 9  max_fea

In [ ]:
# === 입력/타깃 (전처리 완료 가정) ===
X = df.drop('Survived', axis=1)
y = df['Survived'].astype(int)

# === 5-Fold CV 설정 ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(accuracy_score)

# === 1) 두 가지 설정 정의 ===
cfg_best = {
    "learning_rate":    0.004574552455359172,
    "n_estimators":     500,
    "max_depth":        5,
    "subsample":        0.5,
    "min_samples_leaf": 15,
    "max_features":     None,
    "random_state":     42,
}
cfg_alt = {
    "learning_rate":    0.02617,
    "n_estimators":     343,
    "max_depth":        3,
    "subsample":        0.99,
    "min_samples_leaf": 7,
    "max_features":     None,
    "random_state":     42,
}

models = {
    "GBDT_best": GradientBoostingClassifier(**cfg_best),
    "GBDT_alt":  GradientBoostingClassifier(**cfg_alt),
}

# === 2) 교차검증으로 공정 비교 (train/val 평균과 표준편차, gap까지) ===
rows = {}
for name, clf in models.items():
    scores = cross_validate(
        clf, X, y, cv=cv, scoring=scorer,
        return_train_score=True, n_jobs=-1
    )
    tr, va = scores["train_score"], scores["test_score"]
    rows[name] = {
        "TrainAcc(mean)": tr.mean(), "TrainAcc(std)": tr.std(),
        "ValAcc(mean)":   va.mean(), "ValAcc(std)":  va.std(),
        "Gap(mean)":      (tr - va).mean(),
    }

cv_df = pd.DataFrame(rows).T.sort_values("ValAcc(mean)", ascending=False)
display(cv_df.style.format({
    "TrainAcc(mean)": "{:.4f}", "TrainAcc(std)": "{:.4f}",
    "ValAcc(mean)":   "{:.4f}", "ValAcc(std)":   "{:.4f}",
    "Gap(mean)":      "{:+.4f}",
}))

# === 3) 전체 데이터로 재학습 (최종 모델 보관) ===
final_best = GradientBoostingClassifier(**cfg_best).fit(X, y)
final_alt  = GradientBoostingClassifier(**cfg_alt ).fit(X, y)

# (선택) 재학습 후 훈련 정확도(참고용; 과적합 편향 있음)
from sklearn.metrics import accuracy_score
print("\n[Train accuracy (reference only)]")
print("GBDT_best:", accuracy_score(y, final_best.predict(X)))
print("GBDT_alt :", accuracy_score(y, final_alt.predict(X)))



,TrainAcc(mean),TrainAcc(std),ValAcc(mean),ValAcc(std),Gap(mean)
GBDT_best,0.8690,0.0082,0.8406,0.0174,+0.0284
GBDT_alt,0.8973,0.0046,0.8361,0.0167,+0.0612



[Train accuracy (reference only)]
GBDT_best: 0.8709315375982043
GBDT_alt : 0.8922558922558923


In [ ]:
# test.csv 인 df2로 이제 예측 계시
# 1) df2는 train과 동일한 전처리 + 컬럼 구조라고 가정
X_test = df2.copy()

# 2) 예측 확률 (생존할 확률)
proba = final_best.predict_proba(X_test)[:, 1]

# 3) 예측 라벨 (기본 threshold=0.5)
y_test_pred = (proba >= 0.5).astype(int)

# 4) 제출용 DataFrame
submission = pd.DataFrame({
    "PassengerId": df2_pid.astype(int),
    "Survived": y_test_pred
})

submission.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [ ]:
submission.to_csv('submission.csv', index=False)
print("Saved: submission.csv")

Saved: submission.csv


In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>